# **PACKAGES**

To install spaCy, follow the instructions in https://spacy.io/usage  
Operating System, Platform (**ARM/M1** if you have a Apple M1-M3 chip), Package manager, Hardware, Configurations (**virtual env**), Trained pipelines (**English**, **French**, **Spanish**), Select pipeline for (**accuracy**)

In [1]:
import os, sys, csv, time, re
import pandas as pd, numpy as np, matplotlib.pyplot as plt
import openpyxl
from pickle import load
from datetime import datetime

import spacy
import spacy.cli

In [ ]:
#spacy.cli.download("es_dep_news_trf")
#spacy.cli.download("fr_dep_news_trf")

In [2]:
print(sys.version)

3.11.11 (main, Dec 11 2024, 10:25:04) [Clang 14.0.6 ]


# **QUICK SETUP**

In [2]:
pd.set_option('display.max_rows', None)

In [3]:
cty = "Dominican Republic" #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< change here!
lang = "Spanish" #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< change here!

# **DATA IMPORT**

In [4]:
#print(os.getcwd())
path = os.getcwd() + "/data/countries/" + cty.lower().replace(" ", "_")
print(path)

/Users/julienmhp/Desktop/undp/TargetAssessmentReport/data/countries/dominican_republic


In [5]:
file = [os.path.join(path, f) for f in os.listdir(path) if cty.lower().replace(" ", "_") in f and f.endswith(".xlsx")]
print(file[0])

/Users/julienmhp/Desktop/undp/TargetAssessmentReport/data/countries/dominican_republic/data_dominican_republic_23May25.xlsx


In [6]:
dta = pd.read_excel(file[0], sheet_name = "targets", engine = "openpyxl")

In [43]:
dta.head()

,Country,Target Text,Target Name,Document,Source,Doc,Type
0,Dominican Republic,"Impulsar el desarrollo local, provincial y reg...",Objetivo especifíco 1.1.2,Ley 1- 12 Estrategia Nacional de Desarrollo,https://mepyd.gob.do/mepyd/wp-content/uploads/...,LEND,Other targets
1,Dominican Republic,Establecer mecanismos de participación permane...,Línea de acción 1.1.2.3,Ley 1- 12 Estrategia Nacional de Desarrollo,https://mepyd.gob.do/mepyd/wp-content/uploads/...,LEND,Other targets
2,Dominican Republic,"Promover la calidad de la democracia, sus prin...",Objetivo especifíco 1.3.1,Ley 1- 12 Estrategia Nacional de Desarrollo,https://mepyd.gob.do/mepyd/wp-content/uploads/...,LEND,Other targets
3,Dominican Republic,Consolidar y promover la participación de las ...,Línea de acción 1.3.1.4,Ley 1- 12 Estrategia Nacional de Desarrollo,https://mepyd.gob.do/mepyd/wp-content/uploads/...,LEND,Other targets
4,Dominican Republic,Disminuir la pobreza mediante un efectivo y ef...,Objetivo específico 2.3.3,Ley 1- 12 Estrategia Nacional de Desarrollo,https://mepyd.gob.do/mepyd/wp-content/uploads/...,LEND,Other targets


In [15]:
dta.shape

(195, 7)

# **MODEL**

Main attributes (parameters) of the **spaCy** model for NLP:
- **token** each work or symbol  
- **lemma** root of lowecase token
- **pos** part-of-speech (https://universaldependencies.org/u/pos/)
- **dependency** relations between tokens (https://spacy.io/usage/linguistic-features) (https://universaldependencies.org/u/dep/)
- **entity** grammatical role played in phrase (https://spacy.io/usage/linguistic-features) (https://www.universalner.org/)

For more details: https://spacy.io/api/token  
... or use 'print(spacy.explain("{KEYWORD}"))'

### **Model references**

In [131]:
# POS
print(nlp.get_pipe("tagger").labels)
# ADJ (adjective), ADP (adposition), ADV (adverb), AUX (auxiliary verb), CONJ (conjugation), CCONJ (coordinating conjugation), 
# DET (determiner), INTJ (interjection), NOUN, NUM, PART (particle), PRON (pronoun), PROPN (proper noun), PUNCT (punctuation), 
# SCONJ (subordinating conjugation), SYM (symbol), VERB , X (other/unknown), SPACE (white space)
#print(spacy.explain("SYM"))

KeyError: "[E001] No component 'tagger' found in pipeline. Available names: ['transformer', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer']"

In [128]:
# Dependencies
print(nlp.get_pipe("parser").labels)
# ROOT (root of sentence), bsubj (nominal subject), nsubjpass (passive nominal subject), 
# dobj (direct object), iobj (indirect object), attr (attribute), prep (preposition modifier), 
# pobj (object of a preposition), amod (adjectival modifier), advmod (adverbial modifier), 
# compound (compound noun modifier), aux (auxiliary verb), auxpass (passive auxliary), 
# det (determiner), conj (conjugation), cc (coordinating conjugation), mod (nominal modifier), 
# npadvmod (noun phrase as adverbial modifier), poss (possession modifier), 
# ccomp (clausal complement), xcomp (open clausal complement), mark (marker for subordinate clause)
print(spacy.explain("appos"))

('ROOT', 'acl', 'advcl', 'advmod', 'amod', 'appos', 'aux', 'case', 'cc', 'ccomp', 'compound', 'conj', 'cop', 'csubj', 'dep', 'det', 'expl:impers', 'expl:pass', 'expl:pv', 'fixed', 'flat', 'iobj', 'mark', 'nmod', 'nsubj', 'nummod', 'obj', 'obl', 'parataxis', 'punct', 'xcomp')
appositional modifier


In [130]:
# Entities
print(nlp.get_pipe("ner").labels)
# GPE (country, state, city, ...), 
# NORP (nationality, religious or political groups, ...), 
# FAC (buildings, airports, highways, ...), 
# LAW (doucments)
#print(spacy.explain("FAC"))

KeyError: "[E001] No component 'ner' found in pipeline. Available names: ['transformer', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer']"

In [7]:
if lang == "English":
    lang_cd = "en"; media = "web"; model = "core"
elif lang == "Spanish":
    lang_cd = "es"; media = "news"; model = "dep"
elif lang == "French":
    lang_cd = "fr"; media = "news"; model = "dep"

### **Loading the model**

In [8]:
lang_cd+"_"+model+"_"+media+"_trf"

'es_dep_news_trf'

In [138]:
nlp = spacy.load(lang_cd+"_"+model+"_"+media+"_trf")

In [139]:
corpus = list(nlp.pipe(dta["Target Text"]))

In [140]:
rows = []
for corpus, text in zip(corpus, dta["Target Name"]):
    for token in corpus:
        rows.append({
            "Target Name": text,
            "token": token.text,
            "lemma": token.lemma_,
            "pos": token.pos_,
            "dependency": token.dep_,
            "entity": token.ent_type_
        })

tokens_df = pd.DataFrame(rows)

In [141]:
tokens_df.head()
#tokens_df

,Target Name,token,lemma,pos,dependency,entity
0,Objetivo especifíco 1.1.2,Impulsar,impulsar,VERB,ROOT,
1,Objetivo especifíco 1.1.2,el,el,DET,det,
2,Objetivo especifíco 1.1.2,desarrollo,desarrollo,NOUN,obj,
3,Objetivo especifíco 1.1.2,local,local,ADJ,amod,
4,Objetivo especifíco 1.1.2,",",",",PUNCT,punct,


### **Corrections**

#### **Country-specific changes**

In [12]:
if cty == "Uzbekistan":
    tokens_df[tokens_df["Target Name"] == dta["Target Name"][4]] # 222 "I-IV 0 20%" [...]
    #tokens_df[tokens_df["Target Name"] == dta["Target Name"][33]] # 1637 "Target 31" [√]

#### **Language-soecific changes**

**Spanish**

In [142]:
#wrd = ["double", "halve", "half", "triple", "quadruple", "quarter", "quintuple"]
#lgs = ["act", "acts", "bill", "bills", "regulation", "regulations", "decree", "decrees", "article", "articles", "law", "laws", "recommendation", "recommendations", "bill", "bills", "exco"]
org = ["Cantidad"] # ["target", "targets", "goal", "goals", "objective", "objectives", "figure", "table", "zone", "zones", "strategy", "strategies", "strategic", "plan", "plans", "nt", "phase", "phases", "agenda", "agendas", "policy", "policies", "stage", "stages", "programme", "programmes", "action", "actions", "budget"]
mtr = ["ha", "mm"] #["hectare", "hectares", "cm", "m", "km", "km2", "mile", "miles", "g", "kg", "ton", "tons", "ºc", "oc", "microns", "acre", "acres", "factor", "mw"] # "m"?
#mth = [">", "<", "=", "≥", "≤", "~", "≈"]
#qty = ["thousand", "thousands", "hundred", "hundreds", "million", "millions", "billion", "billions", "trillion", "trillions", "mtco2e", "mtco₂e", "co2", "co2e", "co2eq", "co2-eq"]
#prc = ["%", "percent", "percentage"]
tmp = ["año"] # ["annually", "annum", "year", "years", "monthly", "month", "months", "weekly", "week", "weeks", "daily", "day", "days", "hourly", "hour", "hours", tolower(month.name), tolower(month.abb)]
#mny = ["dollar", "usd", "euros", "eur", "$", "£", "¥", "€"] # "M"?2

if lang_cd == "es":
    # ensures tokens that are just digits, or digits with ("-", ",", "." and "%") are CARD
    tokens_df.loc[tokens_df["lemma"].astype(str).str.match(r"^-?\d+[.,]?\d*%?$", na=False) & 
                    ~tokens_df["lemma"].astype(str).str.match(r"^(19|20)\d{2}$", na=False), "entity"] = "CARD"
    
    # makes sure 4-digit numbers - that are not "appos" or "nummod", e.g., DR's "2010-2030" - are DATE
    tokens_df.loc[tokens_df["lemma"].astype(str).str.match(r"^(19|20)\d{2}$") & 
                    ~tokens_df["dependency"].isin(["appos", "nummod"]), "entity"] = "DATE"
    
    # makes sure that spelled-out numbers are CARD
    tokens_df.loc[(tokens_df["pos"] == "NUM") & (tokens_df["lemma"].astype(str).str.match(r"^[A-Aa-z]+$", na=False)), "entity"] = "CARD"
    
    # make sure "%" is CARD
    tokens_df.loc[(tokens_df["lemma"].str.match(r"^\d+$", na=False) & tokens_df["lemma"].shift(-1) == "%"), "entity"] = "CARD"
    tokens_df.loc[tokens_df["lemma"] == "%", "entity"] = "CARD"
    
    # makes sure "hasta", "para (el)" and "al"(and similar) are considered DATE (e.g., "[hasta/para (el)/al] 2030")
    tokens_df.loc[(tokens_df["pos"] == "ADP") & (tokens_df["dependency"] == "case") & 
                    ((tokens_df["entity"].shift(-1) == "DATE") | 
                     ((tokens_df["pos"].shift(-1) == "DET") & (tokens_df["entity"].shift(-2) == "DATE"))
                    ), "entity"] = "DATE"
    tokens_df.loc[(tokens_df["pos"] == "DET") &
                    (tokens_df["entity"].shift(1) == "DATE") & (tokens_df["entity"].shift(-1) == "DATE"), "entity"] = "DATE"
    
    # makes sure that numbers preceded by "name-terms" (e.g. "target") are not CARD/DATE
    tokens_df.loc[(tokens_df["entity"] == "CARD") & 
                    (tokens_df["lemma"].shift(1).isin(org)), "entity"] = ""
    
    # makes sure that numbers preceded by a title (e.g. "ACE 23") are not CARD/DATE
    tokens_df.loc[(tokens_df["pos"] == "NUM") & 
                    ((tokens_df["lemma"].shift(1).astype(str).str.match(r"^[A-Z0-9_-]+$")) | 
                     (tokens_df["pos"].shift(1) == "PROPN")), "entity"] = ""
    
    # makes sure that time references (e.g., "años") preceded by numbers are DATE
    tokens_df.loc[(tokens_df["lemma"].isin(tmp)) & 
                    (tokens_df["entity"].shift(1) == "CARD"), "entity"] = "DATE"
    tokens_df.loc[(tokens_df["entity"] == "CARD") & 
                (tokens_df["lemma"].shift(-1).isin(tmp)), "entity"] = "DATE"
    
    # makes sure that unit measurements are counted as CARD (e.g., "75,102 ha")
    tokens_df.loc[((tokens_df["lemma"].isin(mtr)) | (tokens_df["dependency"] == "appos"))& 
                    (tokens_df["entity"].shift(1) == "CARD"), "entity"] = "CARD"

In [145]:
# test code chuncks here


In [147]:
tokens_df[tokens_df["entity"] == 'CARD']
#tokens_df[tokens_df["entity"] == 'DATE']
#tokens_df
#tokens_df[tokens_df["Target Name"] == 'Objetivo de Mitigación 1.36']

,Target Name,token,lemma,pos,dependency,entity
1746,Meta 5,25%,25%,SYM,advmod,CARD
2571,Objetivo de Mitigación 1.17,1.17,1.17,NUM,nummod,CARD
2583,Objetivo de Mitigación 1.18,"1,18",1.18,NUM,nummod,CARD
2589,Objetivo de Mitigación 1.19,1.19,1.19,NUM,nummod,CARD
2619,Objetivo de Mitigación 1.20,100,100,NUM,appos,CARD
2620,Objetivo de Mitigación 1.20,%,%,SYM,nmod,CARD
2706,Objetivo de Mitigación 1.24,100,100,NUM,nummod,CARD
2707,Objetivo de Mitigación 1.24,%,%,SYM,appos,CARD
2899,Objetivo de Mitigación 1.34,"75,102",75.102,NUM,nummod,CARD
2900,Objetivo de Mitigación 1.34,ha,ha,NOUN,appos,CARD


**English**

In [14]:
#wrd = ["double", "halve", "half", "triple", "quadruple", "quarter", "quintuple"]
#lgs = ["act", "acts", "bill", "bills", "regulation", "regulations", "decree", "decrees", "article", "articles", "law", "laws", "recommendation", "recommendations", "bill", "bills", "exco"]
#org = ["target", "targets", "goal", "goals", "objective", "objectives", "figure", "table", "zone", "zones", "strategy", "strategies", "strategic", "plan", "plans", "nt", "phase", "phases", "agenda", "agendas", "policy", "policies", "stage", "stages", "programme", "programmes", "action", "actions", "budget"]
#mtr = ["ha", "hectare", "hectares", "cm", "m", "km", "km2", "mile", "miles", "g", "kg", "ton", "tons", "ºc", "oc", "microns", "acre", "acres", "factor", "mw"] # "m"?
#mth = [">", "<", "=", "≥", "≤", "~", "≈"]
#qty = ["thousand", "thousands", "hundred", "hundreds", "million", "millions", "billion", "billions", "trillion", "trillions", "mtco2e", "mtco₂e", "co2", "co2e", "co2eq", "co2-eq"]
#prc = ["%", "percent", "percentage"]
#tme = ["annually", "annum", "year", "years", "monthly", "month", "months", "weekly", "week", "weeks", "daily", "day", "days", "hourly", "hour", "hours", tolower(month.name), tolower(month.abb)]
#mny = ["dollar", "usd", "euros", "eur", "$", "£", "¥", "€"] # "M"?2

if lang == "en":
    # Ensures that the word "by", followed by a number representing a date also is considered as a date
    tokens_df.loc[
        (tokens_df["lemma"] == "by") & 
        (tokens_df["pos"].shift(-1) == "NUM") & 
        (tokens_df["entity"].shift(-1) == "DATE"), 
        "entity"] = tokens_df["entity"].shift(-1)
    # Ensures that numbers preceded by any terms such as "target" are not regarded as numbers
    org = ["target", "targets", "goal", "goals", "objective", "objectives", "figure", "table", 
           "zone", "zones", "strategy", "strategies", "strategic", "plan", "plans", 
           "phase", "phases", "agenda", "agendas", "policy", "policies", "stage", "stages", 
           "programme", "programmes", "action", "actions", "budget"]
    tokens_df.loc[(tokens_df["entity"] == "CARDINAL") & 
                    (tokens_df["lemma"].shift(1).isin(org)), "entity"] = ""

#### **General changes**

In [148]:
# Ensures that if the number part of a target name appears in its target text, 
# that token should be regarded as ORG - not NUM
def target_name_in_text(row):
    target = str(row["Target Name"])
    lemma = str(row["lemma"])
    matches = re.findall(r"\d+[.,]\d+", target)
    lemma_normalized = lemma.replace(",", ".")
    matches_normalized = [m.replace(",", ".") for m in matches]
    for num in matches_normalized:
        if num == lemma_normalized:
            return "ORG"
    
    return row["pos"]

tokens_df["pos"] = tokens_df.apply(target_name_in_text, axis=1)

In [149]:
# Ensures that "GPE", "ORG" and "LAW" are discarded as relevant to the analysis - not quant nor temporal
tokens_df.loc[(tokens_df["pos"] == "ORG") | (tokens_df["pos"] == "LAW") | 
                (tokens_df["pos"] == "GPE") | (tokens_df["pos"] == "LOC"), "entity"] = ""

In [150]:
tokens_df[tokens_df["entity"] == 'CARD']

,Target Name,token,lemma,pos,dependency,entity
1746,Meta 5,25%,25%,SYM,advmod,CARD
2619,Objetivo de Mitigación 1.20,100,100,NUM,appos,CARD
2620,Objetivo de Mitigación 1.20,%,%,SYM,nmod,CARD
2706,Objetivo de Mitigación 1.24,100,100,NUM,nummod,CARD
2707,Objetivo de Mitigación 1.24,%,%,SYM,appos,CARD
2899,Objetivo de Mitigación 1.34,"75,102",75.102,NUM,nummod,CARD
2900,Objetivo de Mitigación 1.34,ha,ha,NOUN,appos,CARD
2928,Objetivo de Mitigación 1.34,5,5,NUM,nummod,CARD
2929,Objetivo de Mitigación 1.34,MM,mm,NOUN,nmod,CARD
2954,Objetivo de Mitigación 1.35,"146,648",146.648,NUM,nummod,CARD


In [151]:
tokens_df[tokens_df["entity"] == 'DATE']

,Target Name,token,lemma,pos,dependency,entity
1508,Objetivo estratégico A,para,para,ADP,case,DATE
1509,Objetivo estratégico A,el,el,DET,det,DATE
1510,Objetivo estratégico A,2010,2010,NOUN,nmod,DATE
1583,Meta 1,Para,para,ADP,case,DATE
1584,Meta 1,el,el,DET,det,DATE
1585,Meta 1,2016,2016,NOUN,obl,DATE
1638,Meta 2,Para,para,ADP,case,DATE
1639,Meta 2,el,el,DET,det,DATE
1640,Meta 2,2016,2016,NOUN,obl,DATE
1661,Meta 3,Para,para,ADP,case,DATE


### **Clean-up**

In [152]:
#tokens_df["entity"].unique()

array(['', 'DATE', 'CARD'], dtype=object)

In [153]:
# Eliminates all tokens the "entity" parameter of which inexists
if lang == "en":
    tokens_df = tokens_df.loc[(tokens_df["entity"] != "")]

In [154]:
# Lumps together consecutive tokens that come from the same entity parameter into a single string
tokens_df["flag"] = (
    (tokens_df["entity"] != tokens_df["entity"].shift()) |
    (tokens_df.index != tokens_df.index.to_series().shift() + 1))
tokens_df["entity_group"] = tokens_df["flag"].cumsum()
tokens_df.drop(columns="flag", inplace=True)

In [155]:
tokens_df["mergeable"] = (tokens_df["entity"] != "") & (tokens_df["entity"] != "O")
tokens_df["merge_group"] = tokens_df["entity_group"].where(tokens_df["mergeable"])

In [156]:
merged = (
    tokens_df.groupby(["Target Name", "merge_group", "entity"], dropna=True)
    .agg({"token": " ".join})
    .reset_index()
)
merged = merged.drop(["merge_group"], axis = 1)

In [157]:
# ensures there are no spaces between a number and "%"
merged["token"] = merged["token"].str.replace(r"(\d+)\s+%", r"\1%", regex=True)

In [158]:
merged

,Target Name,entity,token
0,Acción para el Empoderamiento Climatico 4.1,DATE,Al 2030
1,Acción para el Empoderamiento Climatico 4.12,DATE,Al 2022
2,Acción para el Empoderamiento Climatico 4.13,DATE,Al 2030
3,Acción para el Empoderamiento Climatico 4.14,DATE,Al 2022
4,Acción para el Empoderamiento Climatico 4.2,DATE,Al 2030
5,Acción para el Empoderamiento Climatico 4.23,DATE,Al 2021
6,Acción para el Empoderamiento Climatico 4.3,DATE,Al 2030
7,Acción para el Empoderamiento Climatico 4.4,DATE,Al 2030
8,Acción para el Empoderamiento Climatico 4.5,DATE,Para 2024
9,Acción para el Empoderamiento Climatico 4.5,CARD,200


In [159]:
# Creates a list of time-bound terms per target
dates = (
    merged[merged["entity"] == "DATE"]
    .groupby("Target Name")["token"]
    .apply(lambda x: "; ".join(x))
    .reset_index(name="dates")
)

In [162]:
dates

,Target Name,dates
0,Acción para el Empoderamiento Climatico 4.1,Al 2030
1,Acción para el Empoderamiento Climatico 4.12,Al 2022
2,Acción para el Empoderamiento Climatico 4.13,Al 2030
3,Acción para el Empoderamiento Climatico 4.14,Al 2022
4,Acción para el Empoderamiento Climatico 4.2,Al 2030
5,Acción para el Empoderamiento Climatico 4.23,Al 2021
6,Acción para el Empoderamiento Climatico 4.3,Al 2030
7,Acción para el Empoderamiento Climatico 4.4,Al 2030
8,Acción para el Empoderamiento Climatico 4.5,Para 2024
9,Meta 1,Para el 2016


In [160]:
# Creates a list of quantitative terms per target
quants = (
    merged[merged["entity"] != "DATE"]
    .groupby("Target Name")["token"]
    .apply(lambda x: "; ".join(x))
    .reset_index(name="quants")
)

In [161]:
quants

,Target Name,quants
0,Acción para el Empoderamiento Climatico 4.5,200
1,Elementos Transversales 5.1,nueve
2,Medidas de adaptación 2.4,dos
3,Meta 5,25%
4,Objetivo de Mitigación 1.20,100%
5,Objetivo de Mitigación 1.24,100%
6,Objetivo de Mitigación 1.34,"75,102 ha; 5 MM"
7,Objetivo de Mitigación 1.35,"146,648 Ha; 2.2 MM"
8,Objetivo de Mitigación 1.36,"15,000 ha; 43,750 ha"
9,Objetivo de Mitigación 1.41,"30,000 ha"


In [163]:
condens = pd.merge(dates, quants, on="Target Name", how="outer")

In [164]:
condens[["dates", "quants"]] = condens[["dates", "quants"]].fillna("")

In [165]:
condens

,Target Name,dates,quants
0,Acción para el Empoderamiento Climatico 4.1,Al 2030,
1,Acción para el Empoderamiento Climatico 4.12,Al 2022,
2,Acción para el Empoderamiento Climatico 4.13,Al 2030,
3,Acción para el Empoderamiento Climatico 4.14,Al 2022,
4,Acción para el Empoderamiento Climatico 4.2,Al 2030,
5,Acción para el Empoderamiento Climatico 4.23,Al 2021,
6,Acción para el Empoderamiento Climatico 4.3,Al 2030,
7,Acción para el Empoderamiento Climatico 4.4,Al 2030,
8,Acción para el Empoderamiento Climatico 4.5,Para 2024,200
9,Elementos Transversales 5.1,,nueve


# **Saving results**

In [167]:
dta.drop(["Country", "Target Text", "Document", "Source", "Convention", "Doc", "Type"], axis = 1, inplace = True, errors = 'ignore')

In [168]:
final = pd.merge(dta, condens, how = "left")

In [170]:
final = final.fillna("")

In [171]:
final

,Target Name,dates,quants
0,Objetivo especifíco 1.1.2,,
1,Línea de acción 1.1.2.3,,
2,Objetivo especifíco 1.3.1,,
3,Línea de acción 1.3.1.4,,
4,Objetivo específico 2.3.3,,
5,Línea de acción 2.3.3.4,,
6,Objetivo específico 2.4.1,,
7,Línea de acción 2.4.1.2,,
8,Línea de acción 2.4.1.3,,
9,Línea de acción 2.4.1.4,,


In [172]:
final.to_excel(path+"/"+cty+"_quantitative_"+datetime.today().strftime("%d%b%y").lstrip("0")+".xlsx", sheet_name = "Quantitative Terms", index=False)